# Prompt Builder Basics

This notebook explains `FewShotPromptBuilder` in order: main input, generated output, and practical benefits.


## 1. Input: purpose, cautions, and examples

Define what you want, constraints, and a few representative examples.


In [1]:
from openaivec import FewShotPromptBuilder

base_prompt = (
    FewShotPromptBuilder()
    .purpose("Classify fruit names into color and taste notes.")
    .caution("Return concise English.")
    .caution("Use one sentence.")
    .example(
        input_value="apple",
        output_value="Red or green fruit with a sweet-tart taste.",
    )
    .example(
        input_value="lemon",
        output_value="Yellow citrus fruit with a sour taste.",
    )
    .build()
)


## 2. Output: reusable prompt string

`build()` returns a prompt string you can pass to response APIs.


In [2]:
print(base_prompt)


<Prompt>
  <Purpose>Classify fruit names into color and taste notes.</Purpose>
  <Cautions>
    <Caution>Return concise English.</Caution>
    <Caution>Use one sentence.</Caution>
  </Cautions>
  <Examples>
    <Example>
      <Input>apple</Input>
      <Output>Red or green fruit with a sweet-tart taste.</Output>
    </Example>
    <Example>
      <Input>lemon</Input>
      <Output>Yellow citrus fruit with a sour taste.</Output>
    </Example>
  </Examples>
</Prompt>


## 3. Structured examples (optional)

When output shape matters, provide Pydantic models as examples.


In [3]:
from pydantic import BaseModel, Field


class FruitProfile(BaseModel):
    color: str = Field(description="Main color")
    taste: str = Field(description="Primary taste")


structured_prompt = (
    FewShotPromptBuilder()
    .purpose("Extract fruit profile with fixed fields.")
    .caution("taste must be one of: sweet, sour, bitter, umami.")
    .example(
        input_value="banana",
        output_value=FruitProfile(color="yellow", taste="sweet"),
    )
    .example(
        input_value="grapefruit",
        output_value=FruitProfile(color="orange", taste="bitter"),
    )
    .build()
)

print(structured_prompt)


<Prompt>
  <Purpose>Extract fruit profile with fixed fields.</Purpose>
  <Cautions>
    <Caution>taste must be one of: sweet, sour, bitter, umami.</Caution>
  </Cautions>
  <Examples>
    <Example>
      <Input>banana</Input>
      <Output>{"color":"yellow","taste":"sweet"}</Output>
    </Example>
    <Example>
      <Input>grapefruit</Input>
      <Output>{"color":"orange","taste":"bitter"}</Output>
    </Example>
  </Examples>
</Prompt>


## 4. Improve and explain (optional)

`improve()` refines your prompt with an LLM, and `explain()` shows each improvement step as a diff. This example uses `gpt-5.2`.


In [4]:
import os

if os.getenv("OPENAI_API_KEY") or os.getenv("AZURE_OPENAI_BASE_URL"):
    improved_builder = (
        FewShotPromptBuilder()
        .purpose("Classify fruit names into color and taste notes.")
        .caution("Return concise English.")
        .caution("Use one sentence.")
        .example(
            input_value="apple",
            output_value="Red or green fruit with a sweet-tart taste.",
        )
        .example(
            input_value="lemon",
            output_value="Yellow citrus fruit with a sour taste.",
        )
        .improve(model_name="gpt-5.2")
    )
    improved_builder.explain()
    improved_prompt = improved_builder.build()
else:
    print("Skip improve/explain: set OPENAI_API_KEY or Azure environment variables.")


=== Iteration 1 ===

Instruction: Issue addressed (purpose ambiguity): The purpose "Classify fruit names into color and taste notes" is underspecified about the expected output format (e.g., whether to include multiple possible colors, whether to mention category like citrus, and how to behave when a fruit has variable colors). This ambiguity can cause inconsistent outputs across examples and reduce task reliability. The purpose is refined to explicitly require an English, single-sentence description that includes color(s) and taste notes, preserving the original intent.
--- before
+++ after
@@ -1,5 +1,5 @@
 <Prompt>
-  <Purpose>Classify fruit names into color and taste notes.</Purpose>
+  <Purpose>Given a fruit name, output a single concise English sentence describing the fruit’s typical color(s) and its main taste notes.</Purpose>
   <Cautions>
     <Caution>Return concise English.</Caution>
     <Caution>Use one sentence.</Caution>
=== Iteration 2 ===

Instruction: Issue addressed (

In [5]:
print(improved_prompt)


<Prompt>
  <Purpose>Given a fruit name, output a single concise English sentence describing the fruit’s typical color(s) and its main taste notes.</Purpose>
  <Cautions>
    <Caution>Write exactly one concise English sentence (no lists or extra commentary).</Caution>
    <Caution>If multiple colors are common, mention the most typical ones (e.g., "red or green").</Caution>
    <Caution>If the fruit is unknown or the color/taste is highly variable, say "Color varies" and give the most common taste profile.</Caution>
  </Cautions>
  <Examples>
    <Example>
      <Input>apple</Input>
      <Output>Red or green fruit with a sweet-tart taste.</Output>
    </Example>
    <Example>
      <Input>lemon</Input>
      <Output>Yellow citrus fruit with a sour taste.</Output>
    </Example>
    <Example>
      <Input>banana</Input>
      <Output>Yellow fruit with a sweet, creamy taste.</Output>
    </Example>
    <Example>
      <Input>strawberry</Input>
      <Output>Red fruit with a sweet-tart ta

## 5. Benefits

**Main input**
- Prompt purpose (`purpose`)
- Constraints (`caution`)
- Example pairs (`example`)

**Main output**
- Reusable prompt string (`build`)
- Optionally refined prompt and diff history (`improve` + `explain`)

**Why this helps**
- Makes prompt design explicit and reviewable
- Reduces ambiguity by encoding examples in code
- Reuses the same prompt across notebooks and production paths
